In [31]:
import sys
import json
sys.path.append('..')

from src.common import *
from model_performances import *
from copy import deepcopy

import pandas as pd

EVERYTHING_POSTFIX = 'everything'

In [32]:
def to_df(results_all, model_name, answer_type, prompt_type, subs=WITHOUT_RANDOM_SUB):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for question_category in QUESTION_CATEGORIES + [ALL_CATEGORIES_KEY]:
            for ramifications in RAMIFICATION_TYPES:
                index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
                data_columns = {}
                for domain in DOMAIN_NAMES + [ALL_DOMAINS_KEY]:
                    instance = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain,
                          answer_type, subs)
                    if len(instance) == 0:
                        data_columns[domain] = None
                    elif not len(instance) == 1: 
                        raise ValueError(f'len(instance) == {len(instance)}')
                    else:
                        data_columns[domain] = instance[0][SK_RESULT]
                data.append(data_columns)
    return pd.DataFrame(data, index = index)

# def df_to_latex_table(df):
#         latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
#         return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

# def assemble_table(results_all, answer_type, domain, score_key=None):
#     latex_table_all = ''
#     with open('latex_table_template/top') as f:
#         latex_table_all += f.read() + '\n'
#     latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
#     with open('latex_table_template/bottom') as f:
#         latex_table_all += f.read()
#     
#     caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
#     latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
#     
#     return latex_table_all


In [33]:
def collect_results_all(answer_response_type):
    results_all = []
    for domain, plan_length, question_category, ramifications, random_sub, model_name, prompt_type in for_loop_it():
        dir = results_data_path(answer_response_type, domain, plan_length, question_category, ramifications, random_sub, model_name, prompt_type)
        path = os.path.join(dir, RESULTS_FILE_NAME)
        if os.path.exists(path):
            with open(path) as f:
                results_all.append(json.load(f))
    return results_all


In [39]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = F1_SCORE_KEY)
results_all = collect_results_all(tf_answer_type(score_key = F1_SCORE_KEY))
# results_all = open_jsonl(os.path.join(STATISTICS_PATH, save_stats_file(answer_type, score_key)))


100%|██████████| 16128/16128 [00:01<00:00, 12787.85it/s]


In [35]:
model_name = 'gemma-2b-it'
prompt_type = 'few_shot_1'
df = to_df(results_all, model_name, answer_type, prompt_type)

In [40]:
answer_type_ext

'true_false_answer.f1'

In [36]:
df

,blocksworld,depots,driverlog,goldminer,grippers,logistics,miconic,mystery,npuzzle,satellite,spanner,visitall,zenotravel,all_domains
"(1, object_tracking, R, T/F)",0.057143,0.056180,0.000000,0.019231,0.071429,0.033333,0.006098,0.033333,0.000000,0.035176,0.076190,0.017391,0.005747,0.031156
"(1, object_tracking, No R, T/F)",0.042857,0.050505,0.000000,0.000000,0.071429,0.016667,0.012195,0.016667,0.000000,0.030151,0.042857,0.017391,0.005747,0.022500
"(1, fluent_tracking, R, T/F)",0.225926,0.228395,0.076271,0.158683,0.233333,0.205556,0.066667,0.311111,0.055556,0.094444,0.206250,0.131148,0.211111,0.161350
"(1, fluent_tracking, No R, T/F)",0.222222,0.200000,0.047009,0.115044,0.228571,0.205556,0.084211,0.311111,0.088889,0.105556,0.159375,0.140449,0.218519,0.155067
"(1, state_tracking, R, T/F)",0.500000,0.277778,0.000000,NaN,0.450000,0.500000,0.250000,0.500000,0.500000,0.200000,0.400000,NaN,0.500000,0.405000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(19, numerical_reasoning, No R, T/F)",0.100000,0.050000,0.040000,0.005000,0.060000,0.070000,0.030000,0.040000,0.020000,0.035000,0.025000,0.011111,0.115000,0.046512
"(19, hallucination, R, T/F)",0.000000,0.088889,0.000000,0.014286,0.033333,0.020000,0.000000,0.020000,0.000000,0.050000,0.021429,0.009259,0.016667,0.019897
"(19, hallucination, No R, T/F)",0.025000,0.050000,0.016667,0.000000,0.050000,0.070000,0.000000,0.040000,0.033333,0.050000,0.035714,0.018519,0.066667,0.033801
"(19, all_categories, R, T/F)",0.124865,0.128655,0.062147,0.035448,0.118519,0.134615,0.031079,0.107692,0.045503,0.090090,0.115789,0.049399,0.112953,0.087183


In [41]:
latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
os.makedirs(os.path.join(STATISTICS_PATH, 'tables_2'), exist_ok=True)
with open(os.path.join(STATISTICS_PATH, 'tables_2', f'{model_name}.{prompt_type}{answer_type_ext}.tex'), 'w') as f:
    f.write(latex_table)

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_40902/2791806203.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
